In [1]:
import numpy as np
import pandas as pd
import os

import cassava_utils as utils

import torch
from torch.utils.data import DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2

In [2]:
DEVICE = "cuda"

In [3]:
folder_path = "../input/cassava-leaf-disease-classification"
models_path = "../input/resnet50-fold"

In [4]:
# get the path of all models
models_list = []
for model in os.listdir(models_path):
    model = os.path.join(models_path, model)
    models_list.append(model)
models_list.sort()

In [5]:
# define albumentations for test data
data_albums = {
    'test': A.Compose([
        A.Resize(height=400, width=400),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2()])}

In [6]:
def create_test_dataloader(root_dir, data_albums):
    test_files = os.listdir(root_dir)
    test_dataset = utils.CassavaTestDataset(test_files, root_dir, albums=data_albums["test"])
    test_dataloader = {"test": DataLoader(test_dataset, batch_size=16, num_workers=8)}
    return test_dataloader

In [7]:
test_dataloader = create_test_dataloader(os.path.join(folder_path, "test_images"), data_albums)

In [8]:
# load all models from model_path
models = {}
for fold, model_path in enumerate(models_list):
    models[fold] = torch.load(model_path)
    models[fold].eval()

In [9]:
def get_inference_df(dataloader, models):
    test_eng = utils.Engine(model=None, optimizer=None, device=DEVICE)
    inference_df = test_eng.ensemble_predict(dataloader["test"], models)
    return inference_df 

In [10]:
inference_df = get_inference_df(test_dataloader, models)

In [11]:
inference_df.to_csv("submission.csv", index=False)